# Read in KDD99 Dataset

In [ ]:
#importing required libraries

import pandas as pd
import numpy as np
import tensorflow as tf
from tensorflow import keras
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn import metrics

from tensorflow.keras.utils import get_file
try:
    path = get_file('kddcup.data_10_percent.gz', origin='http://kdd.ics.uci.edu/databases/kddcup99/kddcup.data_10_percent.gz')
except:
    print('Error downloading')
    raise

print(path)

In [ ]:
df = pd.read_csv(path, header=None)
print('Read {} rows'.format(len(df)))

In [ ]:
# The CSV file has no column heads, so add them
df.columns = ['duration','protocol_type','service','flag','src_bytes','dst_bytes','land','wrong_fragment','urgent','hot',
              'num_failed_logins','logged_in','num_compromised','root_shell', 'su_attempted','num_root','num_file_creations','num_shells',
              'num_access_files','num_outbound_cmds','is_host_login','is_guest_login','count','srv_count','serror_rate','srv_serror_rate',
              'rerror_rate','srv_rerror_rate','same_srv_rate','diff_srv_rate','srv_diff_host_rate','dst_host_count','dst_host_srv_count',
              'dst_host_same_srv_rate','dst_host_diff_srv_rate','dst_host_same_src_port_rate','dst_host_srv_diff_host_rate','dst_host_serror_rate',
              'dst_host_srv_serror_rate','dst_host_rerror_rate','dst_host_srv_rerror_rate','outcome']

# displaying the contents
df.head()

# Data Processing

In [ ]:
# For now, just drop NA's (rows with missing values)
df.dropna(inplace=True, axis=1)
df.shape

In [ ]:
df.dtypes

In [ ]:
df.groupby('outcome')['outcome'].count()

# Encoding numeric and text data 

In [ ]:
# Encode a numeric column as zscores
def encode_numeric_zscores(df, name, mean=None, sd=None):
    if mean is None:
        mean = df[name].mean()
    if sd is None:
        sd = df[name].std()
    df[name] = (df[name] - mean) / sd

# Encode text values to dummy variables(i.e. [1, 0, 0], [0, 1, 0], [0, 0, 1] for red, green, blue)
def encode_text_dummy(df, name):
    dummies = pd.get_dummies(df[name])
    for x in dummies.columns:
        dummy_name = f'{name}-{x}'
        df[dummy_name] = dummies[x]
    df.drop(name, axis=1, inplace=True)

In [ ]:
# Encoding the feature vector
text_col = ['protocol_type', 'service', 'flag', 'land', 'logged_in', 'is_host_login', 'is_guest_login', ]

for i in df.columns:
    if i not in text_col:
        if i != 'outcome':
            encode_numeric_zscores(df, i)
for x in text_col:
    encode_text_dummy(df, x)

In [ ]:
df.dropna(inplace=True, axis=1)
df[0:5]

In [ ]:
df['protocol_type-tcp'].unique()

In [ ]:
normal_mask = df['outcome'] =='normal.'
attack_mask = df['outcome'] != 'normal.'

df.drop('outcome', axis=1, inplace=True)

df_normal = df[normal_mask]
df_attack = df[attack_mask]

print(f'Normal count: {len(df_normal)}')
print(f'Attack count: {len(df_attack)}')

In [ ]:
# This is the numeric feature vector, as it goes to the neural net
x_normal = df_normal.values
x_attack = df_attack.values

In [ ]:
from sklearn.model_selection import train_test_split

x_normal_train, x_normal_test = train_test_split(x_normal, test_size=0.3, random_state=12)

print(f'Normal train count: {len(x_normal_train)}')
print(f'Normal test count: {len(x_normal_test)}')

In [ ]:
x_normal_train.shape, x_normal_test.shape

In [ ]:
# Scaling the dataset
from sklearn.preprocessing import StandardScaler

sc = StandardScaler()

x_normal_train = sc.fit_transform(x_normal_train)
x_normal_test = sc.fit_transform(x_normal_test)
x_normal = sc.fit_transform(x_normal)
x_attack = sc.fit_transform(x_attack)

In [ ]:
x_normal_train = pd.DataFrame(x_normal_train)
x_normal_test = pd.DataFrame(x_normal_test)
x_attack = pd.DataFrame(x_attack)
x_normal = pd.DataFrame(x_normal)